In [22]:
import io
import requests
import pandas as pd
import pyarrow.parquet as pq
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# url = 'https://storage.googleapis.com/uber-data-engineer-project-iris/uber_data.csv'
# response = requests.get(url)
# df = pd.read_csv(io.StringIO(response.text), sep=',')

#### Yellow Taxi Raw Data

In [23]:
df = pd.read_parquet("data/yellow_tripdata_2024-01.parquet")

In [4]:
# inspecting date type and modify to suitable data type
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [54]:
df = df.drop_duplicates().reset_index(drop=True)

# set pk
df['trip_id'] = df.index

In [76]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0,0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0,1
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0,2
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0,3
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN,2964619
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN,2964620
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN,2964621
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN,2964622


In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,1
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,2
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,3
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,4


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

#### location_dim

In [49]:
zones = gpd.read_file('data/taxi_zones/taxi_zones.shp')
zone_lookup = pd.read_csv('data/taxi_zone_lookup.csv')

In [52]:
zone_lookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       264 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [94]:
location_dim = zones.merge(zone_lookup, on='LocationID').drop(columns=['Borough', 'Zone']).reset_index(drop=True)
location_dim.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,service_zone
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19...",EWR
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343...",Boro Zone
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.77 256767.698, 1026495.593 2...",Boro Zone
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20...",Yellow Zone
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.31 144283.336, 936046.565 144...",Boro Zone


In [97]:
location_dim = location_dim.drop_duplicates(subset=['LocationID'])

In [98]:
location_dim.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 260 entries, 0 to 262
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   OBJECTID      260 non-null    int32   
 1   Shape_Leng    260 non-null    float64 
 2   Shape_Area    260 non-null    float64 
 3   zone          260 non-null    object  
 4   LocationID    260 non-null    int32   
 5   borough       260 non-null    object  
 6   geometry      260 non-null    geometry
 7   service_zone  260 non-null    object  
dtypes: float64(2), geometry(1), int32(2), object(3)
memory usage: 16.2+ KB


In [ ]:
# Plot to see the geo of the df
# location_dim.plot(column='borough', legend=True, figsize=(10, 10))
# plt.title('NYC Taxi Zones by Borough')
# plt.show()

#### pickup_datetime_dim

In [77]:
pickup_datetime_dim = df[['tpep_pickup_datetime']].drop_duplicates().reset_index(drop=True)
pickup_datetime_dim['pick_hour'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.hour
pickup_datetime_dim['pick_day'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.day
pickup_datetime_dim['pick_month'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.month
pickup_datetime_dim['pick_year'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.year
pickup_datetime_dim['pick_weekday'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.weekday

# set pk
pickup_datetime_dim['pickup_datetime_id'] = pickup_datetime_dim.index
# show
pickup_datetime_dim.head()

,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,pickup_datetime_id
0,2024-01-01 00:57:55,0,1,1,2024,0,0
1,2024-01-01 00:03:00,0,1,1,2024,0,1
2,2024-01-01 00:17:06,0,1,1,2024,0,2
3,2024-01-01 00:36:38,0,1,1,2024,0,3
4,2024-01-01 00:46:51,0,1,1,2024,0,4


#### dropoff_datetime_dim

In [78]:
dropoff_datetime_dim = df[['tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)
dropoff_datetime_dim['drop_hour'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.hour
dropoff_datetime_dim['drop_day'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.day
dropoff_datetime_dim['drop_month'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.month
dropoff_datetime_dim['drop_year'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.year
dropoff_datetime_dim['drop_weekday'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.weekday

# set pk
dropoff_datetime_dim['dropoff_datetime_id'] = dropoff_datetime_dim.index
# show the table
dropoff_datetime_dim.head()

,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday,dropoff_datetime_id
0,2024-01-01 01:17:43,1,1,1,2024,0,0
1,2024-01-01 00:09:36,0,1,1,2024,0,1
2,2024-01-01 00:35:01,0,1,1,2024,0,2
3,2024-01-01 00:44:56,0,1,1,2024,0,3
4,2024-01-01 00:52:57,0,1,1,2024,0,4


#### rate_code_dim

In [69]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = pd.DataFrame(list(rate_code_type.items()), columns=['rate_code_id', 'rate_code_name'])
rate_code_dim

,rate_code_id,rate_code_name
0,1,Standard rate
1,2,JFK
2,3,Newark
3,4,Nassau or Westchester
4,5,Negotiated fare
5,6,Group ride


#### payment_type_dim

In [65]:
payment_type = {
    1:"credit card",
    2:"case",
    3:"no charge",
    4:"dispute",
    5:"unknown",
    6:"voided trip"
}

payment_type_dim = pd.DataFrame(list(payment_type.items()), columns=['payment_type_id', 'payment_type_name'])
payment_type_dim

,payment_type_id,payment_type_name
0,1,credit card
1,2,case
2,3,no charge
3,4,dispute
4,5,unknown
5,6,voided trip


### Create Fact Table

In [101]:
fact_table = df.merge(rate_code_dim, left_on='RatecodeID', right_on='rate_code_id', how='left') \
               .merge(pickup_datetime_dim, left_on='tpep_pickup_datetime', right_on='tpep_pickup_datetime', how='left') \
               .merge(dropoff_datetime_dim, left_on='tpep_dropoff_datetime', right_on='tpep_dropoff_datetime', how='left') \
               .merge(payment_type_dim, left_on='payment_type', right_on='payment_type_id', how='left') \
               .merge(location_dim, left_on='PULocationID', right_on='LocationID', how='left') \
               .merge(location_dim, left_on='DOLocationID', right_on='LocationID', how='left') \
               [['trip_id','VendorID', 'pickup_datetime_id', 'dropoff_datetime_id','passenger_count', 'trip_distance',
                 'rate_code_id', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
                 'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                  'total_amount', 'congestion_surcharge', 'Airport_fee']]


In [102]:
fact_table

,trip_id,VendorID,pickup_datetime_id,dropoff_datetime_id,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,PULocationID,DOLocationID,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,congestion_surcharge,Airport_fee
0,0,2,0,0,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,22.70,2.5,0.0
1,1,1,1,1,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,18.75,2.5,0.0
2,2,1,2,2,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,31.30,2.5,0.0
3,3,1,3,3,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,17.00,2.5,0.0
4,4,1,4,4,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2964619,2,1575704,1574775,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,21.77,NaN,NaN
2964620,2964620,1,1533123,1574776,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,25.74,NaN,NaN
2964621,2964621,2,1533585,1574777,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,23.97,NaN,NaN
2964622,2964622,2,1534408,1574778,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,33.46,NaN,NaN


In [ ]:
# CREATE OR REPLACE TABLE `data-with-darshil.uber_dataset.tbl_analysis_report` AS (
# SELECT
#   f.VendorID,
#   f.tpep_pickup_datetime,
#   f.tpep_dropoff_datetime,
#   p.passenger_count,
#   td.trip_distance,
#   rc.RatecodeID,
#   f.store_and_fwd_flag,
#   pl.pickup_latitude,
#   pl.pickup_longitude,
#   dl.dropoff_latitude,
#   dl.dropoff_longitude,
#   pt.payment_type,
#   f.fare_amount,
#   f.extra,
#   f.mta_tax,
#   f.tip_amount,
#   f.tolls_amount,
#   f.improvement_surcharge,
#   f.total_amount
# FROM
#   `data-with-darshil.uber_dataset.fact_table` f
#   JOIN `data-with-darshil.uber_dataset.passenger_count_dim` p ON f.passenger_count_id = p.passenger_count_id
#   JOIN `data-with-darshil.uber_dataset.trip_distance_dim` td ON f.trip_distance_id = td.trip_distance_id
#   JOIN `data-with-darshil.uber_dataset.rate_code_dim` rc ON f.rate_code_id = rc.rate_code_id
#   JOIN `data-with-darshil.uber_dataset.pickup_location_dim` pl ON f.pickup_location_id = pl.pickup_location_id
#   JOIN `data-with-darshil.uber_dataset.dropoff_location_dim` dl ON f.dropoff_location_id = dl.dropoff_location_id
#   JOIN `data-with-darshil.uber_dataset.payment_type_dim` pt ON f.payment_type_id = pt.payment_type_id);